###### imports


In [183]:
from math import pow


###### Constants

In [5]:
MAX_AGE=60
CONTRIBUTION_GROWTH_OPTS = [0.09, 0.08, 0.07, 0.06, 0.05]
IP_LIMIT_FACTORS = {
    0: 0.00000,
    1000000: 1.00000,
    2500000: 1.25000,
    4000000: 1.44000,
    7500000: 1.65000,
    10000000: 1.78000
}
OP_LIMIT_FACTORS = {
    0: 0.00000,
    75000: 1.00000,
    100000: 1.28947,
    200000: 1.63063,
    250000: 1.78000
}
HOSPITAL_ACCESS_GROUP_MAPPER = {
    "All": 1,
    "NHIF": 0.5,
    "Excluding Nairobi, Aga Khan, MP Shah": 0.7
}
FAMILY_SIZE_IP_AMOUNT = {
    "M": 1858181,
    "M+1": 3387031,
    "M+2": 3624949,
    "M+3": 3862866,
    "M+4": 4100784,
    "Add Dep": 237917
}
FAMILY_SIZE_OP_AMOUNT = {
    "M": 688011,
    "M+1": 1302399,
    "M+2": 1336010,
    "M+3": 1369621,
    "M+4": 1403231,
    "Add Dep": 33611
}
IP_FAMILY_SIZE_FACTORS = {
    "M": 1,
    "M+1": 1.822766987,
    "M+2": 1.950804805,
    "M+3": 2.078842624,
    "M+4": 2.206880442,
    "Add Dep": 0.1280378181
}
OP_FAMILY_SIZE_FACTORS = {
    "M": 1,
    "M+1": 1.892992605,
    "M+2": 1.941844698,
    "M+3": 1.99069679,
    "M+4": 2.039548883,
    "Add Dep": 0.04885209257
}

HOSPITAL_ACCESS_GROUPS = list(HOSPITAL_ACCESS_GROUP_MAPPER.keys())

IP_LIMITS = list(IP_LIMIT_FACTORS.keys())
OP_LIMITS = list(OP_LIMIT_FACTORS.keys())

##### contribution period

In [3]:
def get_contribution_period(age):
    """
    Get the contribution period options
    Args:
        age (int): a person's age
    Returns:
        contr_preiod_ops (list): Contribution period options
    """
    contr_preiod_ops = list(range(0, MAX_AGE - age +1))[::-1]
    return contr_preiod_ops

###### Calculations

In [1]:
def generate_contribution_rates(**kwargs):
    """
    Perform the necessary calculations based on the inputs
    Args:
        kwargs (dict): input details including:
            age (int)
            contribution_period (int)
            family_size (str)
            ip_benefit_limit (int)
            op_benefit_limit (int)
    Returns:
        (dict): the calculated values
    """
    age = kwargs.get("age")
    contribution_period = kwargs.get("contribution_period")
    family_size = kwargs.get("family_size")
    ip_benefit_limit = kwargs.get("ip_benefit_limit")
    op_benefit_limit = kwargs.get("op_benefit_limit")
    
    period_to_retirement = MAX_AGE - age
    # contribution_period = list(range(0, period_to_retirement+1))[::-1][0]
    interest_rate = 0.1
    medical_inflation_rate = 0.08
    contribution_growth_rate= CONTRIBUTION_GROWTH_OPTS[0]
    discount_factor = (pow((1+interest_rate),(1/12))-1)*12
    salary_factor = (pow((1+contribution_growth_rate),(1/12))-1)*12
    sannuity_factor = ((1+interest_rate)-1)/discount_factor
    discount_growth_factor = (1+interest_rate)/(1+contribution_growth_rate)-1
    contribution_factor = (sannuity_factor * pow((1+contribution_growth_rate), (contribution_period-1))*\
                (pow((1+discount_growth_factor), (contribution_period))-1)/discount_growth_factor)/\
                 pow((1+medical_inflation_rate), period_to_retirement) if contribution_period > 0 else 1
    
    # ip_benefit_limit = IP_LIMITS[1]
    # op_benefit_limit = OP_LIMITS[1]
    
    ip_benefit_limit_factor = IP_LIMIT_FACTORS.get(ip_benefit_limit, 0)
    op_benefit_limit_factor = OP_LIMIT_FACTORS.get(op_benefit_limit, 0)
    hospital_access_group = HOSPITAL_ACCESS_GROUPS[0]
    hospital_access_group_factor = HOSPITAL_ACCESS_GROUP_MAPPER.get(hospital_access_group)

    # family_size = "M+1"
    ip_family_fize_factor = IP_FAMILY_SIZE_FACTORS.get(family_size)
    op_family_fize_factor = OP_FAMILY_SIZE_FACTORS.get(family_size)

    ip_contribution_amount = FAMILY_SIZE_IP_AMOUNT.get("M")*ip_benefit_limit_factor*hospital_access_group_factor*\
    ip_family_fize_factor/contribution_factor

    op_contribution_amount = FAMILY_SIZE_OP_AMOUNT.get("M")*op_benefit_limit_factor*hospital_access_group_factor*\
    op_family_fize_factor/contribution_factor

    total_contribution_mount = ip_contribution_amount + op_contribution_amount
    
    return {
        "age": age,
        "period_to_retirement": period_to_retirement,
        "contribution_period":contribution_period,
        "interest_rate":interest_rate,
        "medical_inflation_rate": medical_inflation_rate,
        "contribution_growth_rate": contribution_growth_rate,
        "discount_factor": discount_factor,
        "salary_factor": salary_factor,
        "sannuity_factor": sannuity_factor,
        "discount_growth_factor": discount_growth_factor,
        "contribution_factor":contribution_factor,
        "ip_benefit_limit": ip_benefit_limit,
        "op_benefit_limit": op_benefit_limit,
        "hospital_access_group": hospital_access_group,
        "hospital_access_group_factor": hospital_access_group_factor,
        "family_size": family_size,
        "ip_family_fize_factor": ip_family_fize_factor,
        "op_family_fize_factor": op_family_fize_factor,
        "ip_contribution_amount": ip_contribution_amount,
        "op_contribution_amount": op_contribution_amount,
        "total_contribution_mount": total_contribution_mount
    }


###### Test data

In [6]:
data = {
    "age": 59,
    "contribution_period": get_contribution_period(59)[0],
    "family_size": "M+1",
    "ip_benefit_limit": IP_LIMITS[1],
    "op_benefit_limit": OP_LIMITS[1],
}
generate_contribution_rates(**data)

{'age': 59,
 'period_to_retirement': 1,
 'contribution_period': 1,
 'interest_rate': 0.1,
 'medical_inflation_rate': 0.08,
 'contribution_growth_rate': 0.09,
 'discount_factor': 0.09568968514684517,
 'salary_factor': 0.08648787979364059,
 'sannuity_factor': 1.0450447176886444,
 'discount_growth_factor': 0.00917431192660545,
 'contribution_factor': 0.9676339978598558,
 'ip_benefit_limit': 1000000,
 'op_benefit_limit': 75000,
 'hospital_access_group': 'All',
 'hospital_access_group_factor': 1,
 'family_size': 'M+1',
 'ip_family_fize_factor': 1.822766987,
 'op_family_fize_factor': 1.892992605,
 'ip_contribution_amount': 3500322.4257951267,
 'op_contribution_amount': 1345963.1824007945,
 'total_contribution_mount': 4846285.608195921}